In [1]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [4]:
# confirm that our new database was created

if 'uk_food' in mongo.list_database_names():
    print("Database 'uk_food' exists.")
else:
    print("Database 'uk_food' does not exist.")


Database 'uk_food' does not exist.


In [5]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [6]:
# List the collections in the 'uk_food' database
collection_list = db.list_collection_names()

# Print the collections
for collection in collection_list:
    print(collection)


In [7]:
# List the collections in the 'uk_food' database
collection_list = db.list_collection_names()

# Print the collections
for collection_name in collection_list:
    print(collection_name)


In [8]:
# Find and display one document from the 'establishments' collection
document = db.establishments.find_one()
pprint(document)


None


In [9]:
# assign the collection to a variable
establishments = db['establishments']

# Part 2: Update the Database


# 1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [11]:
# Define the new restaurant data
new_restaurant = {
    "name": "Penang Flavours",
    "address": "123 Main Street, Greenwich",
    "city": "London",
    "postcode": "SE10 9XYZ",
    "cuisine": "Halal",
    "rating": None
}


In [12]:
# Insert the new restaurant document into the collection
result = db.establishments.insert_one(new_restaurant)

# Print the inserted document ID
print("Inserted document ID:", result.inserted_id)



Inserted document ID: 646e9178fd76d5af1b172610


In [13]:
# Find the newly inserted restaurant document
inserted_restaurant = db.establishments.find_one({"name": "Penang Flavours"})

# Print the inserted restaurant document
pprint(inserted_restaurant)


{'_id': ObjectId('646e9178fd76d5af1b172610'),
 'address': '123 Main Street, Greenwich',
 'city': 'London',
 'cuisine': 'Halal',
 'name': 'Penang Flavours',
 'postcode': 'SE10 9XYZ',
 'rating': None}


# 2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [14]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return specific fields
query = {"BusinessType": "Restaurant/Cafe/Canteen"}
projection = {"BusinessTypeID": 1, "BusinessType": 1, "_id": 0}

result = db.establishments.find(query, projection)

# Print the results
for document in result:
    pprint(document)


In [16]:
# Update the new restaurant with the correct BusinessTypeID

# Define the query to find the new restaurant
query = {"name": "Penang Flavours"}

# Define the update to set the correct BusinessTypeID
update = {"$set": {"BusinessTypeID": "your_business_type_id"}}

# Update the restaurant document
result = db.establishments.update_one(query, update)

# Print the number of documents updated
print("Number of documents updated:", result.modified_count)


Number of documents updated: 1


In [17]:
# Confirm that the new restaurant was updated

# Find the updated restaurant document
updated_restaurant = db.establishments.find_one({"name": "Penang Flavours"})

# Print the updated restaurant document
pprint(updated_restaurant)


{'BusinessTypeID': 'your_business_type_id',
 '_id': ObjectId('646e9178fd76d5af1b172610'),
 'address': '123 Main Street, Greenwich',
 'city': 'London',
 'cuisine': 'Halal',
 'name': 'Penang Flavours',
 'postcode': 'SE10 9XYZ',
 'rating': None}


# 4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [18]:
# Find how many documents have LocalAuthorityName as "Dover"

# Count the number of documents with LocalAuthorityName as "Dover"
dover_query = {"LocalAuthorityName": "Dover"}
dover_count = db.establishments.count_documents(dover_query)
print("Number of documents with LocalAuthorityName as 'Dover':", dover_count)


Number of documents with LocalAuthorityName as 'Dover': 0


In [19]:
# Delete all documents where LocalAuthorityName is "Dover"

dover_query = {"LocalAuthorityName": "Dover"}
delete_result = db.establishments.delete_many(dover_query)

# Print the number of deleted documents
print("Number of documents deleted:", delete_result.deleted_count)



Number of documents deleted: 0


In [20]:
# Check if any remaining documents include Dover

remaining_query = {"LocalAuthorityName": "Dover"}
remaining_count = db.establishments.count_documents(remaining_query)
if remaining_count > 0:
    print("There are remaining documents with LocalAuthorityName as 'Dover'")
else:
    print("No remaining documents with LocalAuthorityName as 'Dover'")


No remaining documents with LocalAuthorityName as 'Dover'


In [21]:
# Check if any documents remain in the collection using find_one
remaining_document = db.establishments.find_one({})
if remaining_document is not None:
    print("There are remaining documents in the collection.")
    pprint(remaining_document)
else:
    print("No remaining documents in the collection.")


There are remaining documents in the collection.
{'BusinessTypeID': 'your_business_type_id',
 '_id': ObjectId('646e9178fd76d5af1b172610'),
 'address': '123 Main Street, Greenwich',
 'city': 'London',
 'cuisine': 'Halal',
 'name': 'Penang Flavours',
 'postcode': 'SE10 9XYZ',
 'rating': None}



# 5. Some of the number values are stored as strings, when they should be stored as numbers.Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [22]:
# Change the data type from String to Decimal for longitude and latitude

from bson.decimal128 import Decimal128

# Convert latitude and longitude fields to decimal numbers
update_query = {"$or": [{"longitude": {"$type": "string"}}, {"latitude": {"$type": "string"}}]}
update_fields = {"longitude": Decimal128("0"), "latitude": Decimal128("0")}

result = db.establishments.update_many(update_query, {"$set": update_fields})

print("Number of documents updated:", result.modified_count)


Number of documents updated: 0


In [24]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])


In [25]:
# Convert RatingValue field to integer numbers
result = db.establishments.update_many(
    {"RatingValue": {"$type": "string"}},
    [{"$set": {"RatingValue": {"$toInt": "$RatingValue"}}}]
)

print("Number of documents updated:", result.modified_count)


Number of documents updated: 0


In [26]:
# Check the data types of coordinates and RatingValue fields
document = db.establishments.find_one({})

latitude = document.get("latitude")
longitude = document.get("longitude")
rating_value = document.get("RatingValue")

if isinstance(latitude, (int, float)) and isinstance(longitude, (int, float)) and isinstance(rating_value, (int, float)):
    print("Coordinates (latitude and longitude) and RatingValue are now numbers.")
else:
    print("Coordinates (latitude and longitude) and RatingValue are not numbers.")


Coordinates (latitude and longitude) and RatingValue are not numbers.
